In [79]:
import pandas as pd
import mysql.connector


cnx = mysql.connector.connect(
    host = "--------",
    user = "--------",
    password = "--------",
    database = "--------"
    )


if cnx.is_connected():
    print("Conectado ao Banco de dados")

itens_pedido = pd.read_sql_query("SELECT * FROM itens_pedido LIMIT 25057;", cnx)
pedido = pd.read_sql_query("SELECT * FROM pedido LIMIT 25056;", cnx)
produto = pd.read_sql_query("SELECT * FROM produto LIMIT 205;", cnx)
vendedores = pd.read_sql_query("SELECT * FROM vendedores LIMIT 1000;", cnx)


cnx.close()


Conectado ao Banco de dados


C:\Users\gabri\AppData\Local\Temp\ipykernel_8420\1828907175.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  itens_pedido = pd.read_sql_query("SELECT * FROM itens_pedido LIMIT 25057;", cnx)
C:\Users\gabri\AppData\Local\Temp\ipykernel_8420\1828907175.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedido = pd.read_sql_query("SELECT * FROM pedido LIMIT 25056;", cnx)
C:\Users\gabri\AppData\Local\Temp\ipykernel_8420\1828907175.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produto = pd.read_sql_query("SELECT * FR

In [80]:
df_list = {"itens_pedido":itens_pedido, "pedido":pedido, "produto":produto, "vendedores":vendedores}

for name, df in df_list.items():
    print(f"A quantidade de dados nulos na tabela {name} é {df.isna().sum().sum()}") 

A quantidade de dados nulos na tabela itens_pedido é 0
A quantidade de dados nulos na tabela pedido é 0
A quantidade de dados nulos na tabela produto é 0
A quantidade de dados nulos na tabela vendedores é 0


In [81]:
itens_pedido.isna().sum()

produto_id        0
pedido_id         0
quantidade        0
valor_unitario    0
valor_total       0
estado            0
frete             0
id_nf             0
dtype: int64

In [82]:
itens_pedido[itens_pedido.pedido_id.isna()]

,produto_id,pedido_id,quantidade,valor_unitario,valor_total,estado,frete,id_nf


Todos os valores nulos do dataframe itens_pedido estão em um único pedido, então irei remover esse registro do banco de dados.

In [83]:
pedido[pedido.vendedor_id.isna()]

,pedido_id,produto_id,vendedor_id,data_compra,total


Já na tabela pedido, as coluna 'vendedor_id' possuí muitos dados nulos, pressuponho que esses pedidos tenham sido feitos pelo site, e por isso não houve um vendedor para intermediar, então irei adicionar a opção de "usuário" na lista de vendedores 

In [84]:
vendedores

,vendedor_id,nome_vendedor
0,1,Ana Duarte
1,2,Daniel Siqueira
2,3,Nádia Oliveira
3,4,Millena Pereira
4,5,Paulo Calanca
5,6,Usuário


In [85]:
lista_produtos = produto['produto_id'].to_list()
outros_produtos = itens_pedido['produto_id'].unique().tolist()
todos_produtos = lista_produtos + outros_produtos
pro = pd.DataFrame({"produto":todos_produtos})

qtd = []

for i in range(len(todos_produtos)):
    qtd.append(1)
    
pro["qtd"] = qtd

In [86]:
pro

,produto,qtd
0,2706,1
1,3451,1
2,3820,1
3,3829,1
4,3865,1
...,...,...
462,41,1
463,36,1
464,55,1
465,17,1


In [87]:
pro = pro.groupby('produto').sum()

In [88]:
len(pro.query('qtd==1'))

57

Existem 57 produtos que aparecem na tabela de itens_pedido, mas não aparecem na tabela produto